In [1]:
import sys
import os

path = "C:/Users/cr-19/OneDrive - Universidad de Antioquia/2024-2/Aplicaciones de la Analítica/Entregas/Analitica_3_Proyecto_1"
os.chdir(path) # setting a Working directory on drive path
sys.path.append(path+'/')  # setting path to read user defined functions

In [2]:
import a_funciones as fn # importar modulo de funciones

In [3]:
from a_funciones import * # importar todas las funciones del archivo

# con esto ya ses pueden utilizar todas las funciones propias

In [4]:
# Cómo trabajar con sql

import sqlite3 as sql
import pandas as pd

In [5]:
con = sql.connect('base.db')  # crea base de datos vacía # para traer los datos de una tabla
cur=con.cursor()  # para ejecutar consultas en la base de datos # para crear una tabla

In [6]:
# es un estandar que sirve para consultar qué tablas tiene mi base de datos

cur.execute("Select name from sqlite_master where type='table'") # consultar bases de datos
cur.fetchall()

[('retiros',), ('general',), ('employee_survey',), ('manager',)]

In [7]:
# llevar datos a la base de datos

retiros = pd.read_csv('C:/Users/cr-19/OneDrive - Universidad de Antioquia/2024-2/Aplicaciones de la Analítica/Entregas/Analitica_3_Proyecto_1/data/db_retirement.csv')
retiros.to_sql('retiros',con,index=False, if_exists='replace') ### para llevar un data frame de pandas a SQL
general = pd.read_csv("C:/Users/cr-19/OneDrive - Universidad de Antioquia/2024-2/Aplicaciones de la Analítica/Entregas/Analitica_3_Proyecto_1/data/db_general.csv")
general.to_sql('general',con,index=False, if_exists='replace')
employee_survey = pd.read_csv("C:/Users/cr-19/OneDrive - Universidad de Antioquia/2024-2/Aplicaciones de la Analítica/Entregas/Analitica_3_Proyecto_1/data/db_employee.csv")
employee_survey.to_sql('employee_survey',con,index=False, if_exists='replace')
manager = pd.read_csv("C:/Users/cr-19/OneDrive - Universidad de Antioquia/2024-2/Aplicaciones de la Analítica/Entregas/Analitica_3_Proyecto_1/data/db_manager.csv")
manager.to_sql('manager',con,index=False, if_exists='replace')

8820

In [8]:
pd.read_sql(""" select * from retiros
""", con=con)

,EmployeeID,Attrition,retirementDate,retirementType,resignationReason
0,2,Yes,2016-10-15,Resignation,Others
1,7,Yes,2016-07-20,Resignation,Stress
2,14,Yes,2016-11-08,Resignation,Others
3,29,Yes,2016-10-15,Resignation,Others
4,31,Yes,2016-02-08,Resignation,Salary
...,...,...,...,...,...
706,4382,Yes,2015-04-18,Resignation,Salary
707,4387,Yes,2015-09-16,Resignation,Others
708,4389,Yes,2015-05-24,Resignation,Others
709,4392,Yes,2015-02-26,Fired,None


In [9]:
# Importante: 2015 no se tiene la info completa entonces esa información no se debe utilizar para un modelo porque no está completa
# 2015 sirve para hacer EDA para saber las causas de los retiros

pd.read_sql("""
select strftime('%Y',retirementDate) as year, count(distinct EmployeeID) as conteo
from retiros
group by year
""", con=con)

,year,conteo
0,2015,61
1,2016,650


In [10]:
pd.read_sql("select * from general", con = con)

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,...,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,InfoDate
0,51,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,1,...,Y,11,8,0,1.0,6,1,0,0,2015-12-31
1,31,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,1,...,Y,23,8,1,6.0,3,5,1,4,2015-12-31
2,32,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,4,...,Y,15,8,3,5.0,2,5,0,3,2015-12-31
3,38,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,3,...,Y,11,8,3,13.0,5,8,7,5,2015-12-31
4,32,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,1,...,Y,12,8,2,9.0,2,6,0,4,2015-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8815,42,Travel_Rarely,Research & Development,5,4,Medical,1,4406,Female,1,...,Y,17,8,1,10.0,5,3,0,2,2016-12-31
8816,29,Travel_Rarely,Research & Development,2,4,Medical,1,4407,Male,1,...,Y,15,8,0,10.0,2,3,0,2,2016-12-31
8817,25,Travel_Rarely,Research & Development,25,2,Life Sciences,1,4408,Male,2,...,Y,20,8,0,5.0,4,4,1,2,2016-12-31
8818,42,Travel_Rarely,Sales,18,2,Medical,1,4409,Male,1,...,Y,14,8,1,10.0,2,9,7,8,2016-12-31


In [11]:
# El primer proceso que debemos hacer es que a la bd general, de los empleados cuáles se retiraron
# Se puede hacer de forma directa porque la bd de Retiros tiene datos únicos
# Si voy a montar un modelo que prediga si los empleados se van a retirar hay que tener en cuenta 2016 y no 2015 porque tiene muy pocos datos

In [12]:
# Tener cuidado porque no se puede tener dos predicciones para el mismo empleado

pd.read_sql("""
select count(distinct EmployeeID) as conteo from general
""", con=con)

,conteo
0,5060


In [13]:
pd.read_sql("""
select InfoDate, count(distinct EmployeeID) as conteo from general group by InfoDate
""", con=con)

,InfoDate,conteo
0,2015-12-31,4410
1,2016-12-31,4410


In [14]:
# Tener cuidado con esta info vs. los retiros porque, por ejemplo,
# yo no podría coger la info de los empleados a corte de dic de 2016 porque esos empleados
# ya no van a estar.

pd.read_sql("""
select InfoDate, count(distinct EmployeeID) as conteo from general group by InfoDate
""", con=con)

,InfoDate,conteo
0,2015-12-31,4410
1,2016-12-31,4410


In [15]:
pd.read_sql("""
select * from employee_survey
""", con=con)

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,DateSurvey
0,1,3.0,4.0,2.0,2015-12-31
1,2,3.0,2.0,4.0,2015-12-31
2,3,2.0,2.0,1.0,2015-12-31
3,4,4.0,4.0,3.0,2015-12-31
4,5,4.0,1.0,3.0,2015-12-31
...,...,...,...,...,...
8815,4406,4.0,1.0,3.0,2016-12-31
8816,4407,4.0,4.0,3.0,2016-12-31
8817,4408,1.0,3.0,3.0,2016-12-31
8818,4409,4.0,1.0,3.0,2016-12-31


In [23]:
# esta bd tiene 5060 registros mientras que la general tiene 8820, es decir, el registro por 'EmployeeID' está repetido

pd.read_sql("""
select count(distinct EmployeeID) as conteo from employee_survey
""", con=con)

,conteo
0,5060


In [25]:
cur.execute("""
create table datos full select * from general a left join employye_survey b on a.EmployeeID = b.EmployeeID
""", con=con)

TypeError: Cursor.execute() takes no keyword arguments